In [1]:
import random
import pandas as pd
import numpy as np
import time

In [2]:
def diff(Rule1, Rule2):
    if Rule1[-1] != Rule2[-1]:
        return -1
    m = len(Rule1)
    count = 0
    for i in range(m-1):
        if Rule1[i] == Rule2[i]:
            count += 1
    return count/m

In [3]:
def sampling(base, ran, e, k = 2):
    

    num = len(base)
    R = []
    list_index = []
    while (len(R)<num*ran/100):
        index = random.randrange(num)
        while (index in list_index):
            index = random.randrange(num)
        # list_index.append(index)
        T = []
        T.append(index)
        for i in range(index-k,index+k+1):
            if i < num:
                if i in list_index:
                    continue
                else:
                    if diff(base[i],base[index]) < 1 - e:
                        T.append(i)
        for i in T:
            temp = 0
            for j in range(len(R)):
                if diff(base[i],R[j]) < 1 - e:
                    continue
                else:
                    temp = 1
            if temp:
                T.remove(i)
        for i in T:
            R.append(base[i])
            list_index.append(i)
    
    return R



In [4]:
def combination(k, n):
    if k == 0 or k == n:
        return 1
    if k == 1:
        return n
    return combination(k - 1, n - 1) + combination(k, n - 1)


def caculateA(base):
    colum = len(base[0])
    row = len(base)
    A = np.zeros((row, combination(4, colum - 1)))

    for r1 in range(row):
        k = [0] * combination(4, colum - 1)
        temp = 0
        for a in range(0, colum - 4):
            for b in range(a + 1, colum - 3):
                for c in range(b + 1, colum - 2):
                    for d in range(c + 1, colum - 1):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r1][d] == base[r2][d]:
                                k[temp] += 1

                        A[r1][temp] = k[temp] / row
                        temp += 1
    print("done A")
    return A


def caculateM(base):
    colum = len(base[0])
    row = len(base)
    M = np.zeros((row, colum - 1))
    for t1 in range(row):
        k = [0] * (colum - 1)
        temp = 0
        for i in range(colum - 1):
            for t2 in range(row):
                if base[t1][i] == base[t2][i] and base[t1][colum - 1] == base[t2][colum - 1]:
                    k[temp] += 1
            M[t1][temp] = k[temp] / row
            temp += 1

    return M


def caculateB(base, A, M):
    colum = len(base[0])
    row = len(base)
    B = np.zeros((row, combination(3, colum - 1)))

    for r in range(row):
        temp = 0
        for a in range(0, colum - 3):
            for b in range(a + 1, colum - 2):
                for c in range(b + 1, colum - 1):
                    B[r][temp] = sum(A[r]) * min(M[r][a], M[r][b], M[r][c])
                    temp += 1
    print("done B")
    return B


def caculateC(base, B):
    colum = len(base[0])
    row = len(base)
    cols = 2 * combination(3, colum - 1)
    C = np.zeros((row, cols))

    for r1 in range(row):
        temp = 0
        for i in range(2):
            for a in range(0, (colum - 3)):
                for b in range(a + 1, (colum - 2)):
                    for c in range(b + 1, (colum - 1)):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r2][colum - 1] == i:
                                C[r1][temp] += B[r2][temp % combination(3, colum - 1)]
                        # print(temp,":",temp//combination(3,colum-1))
                        temp += 1
    print("done C")
    return C

In [5]:
def FISA(base, C, list):
    colum = len(base[0])
    row = len(base)

    cols = combination(3, (colum - 1))
    C0 = [0] * cols
    C1 = [0] * cols

    t = 0
    for a in range(0, colum - 3):
        for b in range(a + 1, colum - 2):
            for c in range(b + 1, colum - 1):
                for r in range(row-1):
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 0:
                        C0[t] = C[r][t + 0 * cols]
                        # break
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 1:
                        C1[t] = C[r][t + 1 * cols]
                        # break
                t += 1
    #print(t)

    D0 = max(C0) + min(C0)
    D1 = max(C1) + min(C1)

    #print(D0, max(C0), min(C0))
    #print(D1, max(C1), min(C1))
    #print(D2, max(C2), min(C2))
    if D0 > 3*D1:
        return 0, D0/(D0+D1)
    else:
        return 1, D1/(D0+D1)

In [6]:
def Acc(A,B):
    result = 0

    for i in range(len(A)):
        if int(A[i]) - int(B[i]) == 0:
            result += 1

    return round(result*100/len(A), 2)


def Tprecision(Pre,Act):
    result = 0
    TP = 0
    FP = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 1 and int(Act[i]) == 0:
            FP +=1

    #return str(TP) +  " : " + str(TP+FP)

    if TP:
        return round(100*TP/(TP+FP),2)
    else:
        if FP:
            return 0
        else:
            return None


def Trecall(Pre,Act):
    result = 0
    TP = 0
    FN = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 0 and int(Act[i]) == 1:
            FN +=1

    #return str(TP) +  " : " + str(TP+FN)

    if TP:
        return round(100*TP/(TP+FN),2)
    else:
        if FN:
            return 0
        else:
            return None

# listAcc = []
# listPre = []
# listRe = []
# timeTest = []
# timeUpdate = []
# listRank = []
# res = []

def testAccuracy(base,Te,C):
    test = Te
    X = np.zeros(len(test))
    ddd = np.zeros(len(test))
    X_test = np.array(test).T[-1]
    for i in range(len(test)):
        X[i], ddd[i] = FISA(base, C, test[i])
        # listRank.append(ddd[i])
        #print(test[i])
    # for dd in ddd:
    #     print(dd)
    # listRank.append(ddd)

    #print(X)
    #print(X_test)
    # res.append(X)
    # listAcc.append(Acc(X,X_test))
    # listPre.append(Tprecision(X,X_test))
    # listRe.append(Trecall(X,X_test))

    return Acc(X,X_test)

In [7]:
def scenario_FKGS(ran, e):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    train = traindf.values.tolist()
    test = testdf.values.tolist()


    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        start = time.time()
        base = sampling(ran=ran, base=train, e=e)
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)


    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [8]:
def scenario_random(ran):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    test = testdf.values.tolist()

    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        
        start = time.time()
        train = traindf.sample(n=int(len(traindf)*ran/100))
        base = train.values.tolist()
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)

    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [9]:
scenario_FKGS(10, 0.1)

len base 0: 1107
done A
done B
done C
len base 1: 1109
done A
done B
done C
len base 2: 1105
done A
done B
done C
len base 3: 1110
done A
done B
done C
len base 4: 1107
done A
done B
done C
len base 5: 1105
done A
done B
done C
len base 6: 1107
done A
done B
done C
len base 7: 1109
done A
done B
done C
len base 8: 1109
done A
done B
done C
len base 9: 1105
done A
done B
done C
sampling_time: [0.21645736694335938, 0.1992778778076172, 0.19667935371398926, 0.19585633277893066, 0.19971585273742676, 0.19886302947998047, 0.19680118560791016, 0.20203089714050293, 0.2017052173614502, 0.2087709903717041]
train_time: [30.15041160583496, 30.158881187438965, 30.157491445541382, 29.23056721687317, 29.494225025177002, 29.50538730621338, 29.823099851608276, 30.34107732772827, 30.190476417541504, 28.43437147140503]
test_time: [30.039283275604248, 29.532310247421265, 29.602198123931885, 29.300920248031616, 29.30296754837036, 29.397674083709717, 29.412065267562866, 29.48505473136902, 29.439715147018433,

In [10]:
scenario_FKGS(10, 0.2)

len base 0: 1108
done A
done B
done C
len base 1: 1109
done A
done B
done C
len base 2: 1106
done A
done B
done C
len base 3: 1105
done A
done B
done C
len base 4: 1106
done A
done B
done C
len base 5: 1106
done A
done B
done C
len base 6: 1105
done A
done B
done C
len base 7: 1106
done A
done B
done C
len base 8: 1108
done A
done B
done C
len base 9: 1107
done A
done B
done C
sampling_time: [0.21385407447814941, 0.2091386318206787, 0.2118091583251953, 0.2079308032989502, 0.20821714401245117, 0.21163368225097656, 0.20819520950317383, 0.20385050773620605, 0.2108609676361084, 0.20887303352355957]
train_time: [24.994093418121338, 24.964593648910522, 24.841723680496216, 24.799553394317627, 24.83319878578186, 24.75544548034668, 24.733433961868286, 24.54220724105835, 25.322019338607788, 25.263640642166138]
test_time: [28.085859298706055, 28.149964094161987, 28.001395225524902, 28.01274561882019, 27.980029821395874, 28.02479314804077, 28.010967254638672, 27.910860300064087, 28.25271487236023,

In [11]:
scenario_FKGS(10, 0.3)

len base 0: 1105
done A
done B
done C
len base 1: 1105
done A
done B
done C
len base 2: 1106
done A
done B
done C
len base 3: 1105
done A
done B
done C
len base 4: 1106
done A
done B
done C
len base 5: 1105
done A
done B
done C
len base 6: 1106
done A
done B
done C
len base 7: 1105
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1105
done A
done B
done C
sampling_time: [0.2428574562072754, 0.23398804664611816, 0.24809002876281738, 0.23810815811157227, 0.2389991283416748, 0.23972010612487793, 0.2410566806793213, 0.2501406669616699, 0.24115443229675293, 0.2427065372467041]
train_time: [25.312384605407715, 25.39884352684021, 25.792176723480225, 24.674529552459717, 25.359079360961914, 24.88921880722046, 24.73135757446289, 24.354552030563354, 25.471246242523193, 25.42361283302307]
test_time: [28.303972959518433, 29.36228060722351, 28.314780712127686, 28.14744472503662, 28.327236890792847, 28.17585825920105, 28.13100814819336, 28.040948152542114, 28.393163442611694, 28

In [12]:
scenario_FKGS(10, 0.4)

len base 0: 1105
done A
done B
done C
len base 1: 1106
done A
done B
done C
len base 2: 1105
done A
done B
done C
len base 3: 1106
done A
done B
done C
len base 4: 1105
done A
done B
done C
len base 5: 1105
done A
done B
done C
len base 6: 1106
done A
done B
done C
len base 7: 1106
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1106
done A
done B
done C
sampling_time: [0.278228759765625, 0.27824926376342773, 0.27315330505371094, 0.2771925926208496, 0.27413225173950195, 0.27230262756347656, 0.27013659477233887, 0.27128052711486816, 0.269176721572876, 0.2681393623352051]
train_time: [28.027309894561768, 26.945993423461914, 26.987135887145996, 27.411988019943237, 27.45763087272644, 26.68085503578186, 26.42263889312744, 26.284483432769775, 26.935714721679688, 27.67062020301819]
test_time: [29.061944007873535, 28.765429258346558, 28.85849094390869, 28.887073040008545, 28.867124319076538, 28.608017206192017, 28.64030408859253, 28.58454132080078, 28.723868370056152, 28

In [13]:
scenario_FKGS(10, 0.5)

len base 0: 1105
done A
done B
done C
len base 1: 1106
done A
done B
done C
len base 2: 1106
done A
done B
done C
len base 3: 1106
done A
done B
done C
len base 4: 1106
done A
done B
done C
len base 5: 1106
done A
done B
done C
len base 6: 1105
done A
done B
done C
len base 7: 1105
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1106
done A
done B
done C
sampling_time: [0.282120943069458, 0.2772700786590576, 0.2921919822692871, 0.2662501335144043, 0.2762753963470459, 0.2762928009033203, 0.27528882026672363, 0.2842531204223633, 0.2801699638366699, 0.27692627906799316]
train_time: [27.254619121551514, 26.749333381652832, 27.310678958892822, 27.069822311401367, 27.781017303466797, 26.955297231674194, 26.542654037475586, 27.62627148628235, 26.33047842979431, 27.683349609375]
test_time: [28.80165457725525, 28.724598169326782, 28.831241607666016, 28.72976064682007, 28.879103422164917, 28.77237367630005, 28.643242597579956, 28.84608793258667, 28.621464252471924, 29.0538

In [14]:
scenario_FKGS(15, 0.1)

len base 0: 1662
done A
done B
done C
len base 1: 1660
done A
done B
done C
len base 2: 1657
done A
done B
done C
len base 3: 1660
done A
done B
done C
len base 4: 1662
done A
done B
done C
len base 5: 1658
done A
done B
done C
len base 6: 1661
done A
done B
done C
len base 7: 1661
done A
done B
done C
len base 8: 1662
done A
done B
done C
len base 9: 1661
done A
done B
done C
sampling_time: [0.4580843448638916, 0.4498775005340576, 0.45841479301452637, 0.4440169334411621, 0.44898200035095215, 0.44896817207336426, 0.44390010833740234, 0.4579794406890869, 0.4440131187438965, 0.4580371379852295]
train_time: [67.73563432693481, 65.3609266281128, 67.20168495178223, 67.11505222320557, 67.65308451652527, 67.74325227737427, 64.85520052909851, 66.87885880470276, 68.12325954437256, 67.06935214996338]
test_time: [44.64062476158142, 44.25401997566223, 44.6526882648468, 44.212403535842896, 44.34872102737427, 44.42707896232605, 44.0101113319397, 44.32297730445862, 44.501978397369385, 44.333136320114

In [15]:
scenario_FKGS(15, 0.2)

len base 0: 1659
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1659
done A
done B
done C
len base 3: 1658
done A
done B
done C
len base 4: 1659
done A
done B
done C
len base 5: 1659
done A
done B
done C
len base 6: 1658
done A
done B
done C
len base 7: 1660
done A
done B
done C
len base 8: 1659
done A
done B
done C
len base 9: 1658
done A
done B
done C
sampling_time: [0.5001943111419678, 0.48009181022644043, 0.48201990127563477, 0.4741485118865967, 0.48897337913513184, 0.4891631603240967, 0.5007247924804688, 0.483905553817749, 0.49770236015319824, 0.5056767463684082]
train_time: [55.18187403678894, 56.15448760986328, 55.73843741416931, 55.60094499588013, 56.467774629592896, 55.69015693664551, 56.286903619766235, 56.50494146347046, 55.61375880241394, 58.45532250404358]
test_time: [42.590678215026855, 42.572953939437866, 42.66902446746826, 42.54287123680115, 42.716797828674316, 43.59524583816528, 43.38460683822632, 43.24965262413025, 43.24992084503174, 43.3663179

In [16]:
scenario_FKGS(15, 0.3)

len base 0: 1658
done A
done B
done C
len base 1: 1658
done A
done B
done C
len base 2: 1657
done A
done B
done C
len base 3: 1657
done A
done B
done C
len base 4: 1659
done A
done B
done C
len base 5: 1659
done A
done B
done C
len base 6: 1657
done A
done B
done C
len base 7: 1657
done A
done B
done C
len base 8: 1659
done A
done B
done C
len base 9: 1657
done A
done B
done C
sampling_time: [0.5709614753723145, 0.584564208984375, 0.5691483020782471, 0.5704855918884277, 0.5807046890258789, 0.5676157474517822, 0.5649423599243164, 0.5618369579315186, 0.5605976581573486, 0.5704455375671387]
train_time: [59.04718637466431, 57.93484306335449, 58.42372751235962, 57.594711780548096, 57.91272449493408, 57.94962930679321, 58.65288496017456, 58.82412552833557, 57.65539526939392, 57.27959394454956]
test_time: [44.130075216293335, 43.769439458847046, 43.827399015426636, 43.70570349693298, 43.54877853393555, 43.6092312335968, 43.8711953163147, 43.76757001876831, 43.50060677528381, 43.50636649131775

In [17]:
scenario_FKGS(15, 0.4)

len base 0: 1657
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1657
done A
done B
done C
len base 3: 1658
done A
done B
done C
len base 4: 1657
done A
done B
done C
len base 5: 1657
done A
done B
done C
len base 6: 1657
done A
done B
done C
len base 7: 1657
done A
done B
done C
len base 8: 1657
done A
done B
done C
len base 9: 1658
done A
done B
done C
sampling_time: [0.6473352909088135, 0.6205217838287354, 0.622450590133667, 0.611762523651123, 0.6179211139678955, 0.6159896850585938, 0.6234035491943359, 0.6140553951263428, 0.6320586204528809, 0.6270668506622314]
train_time: [63.156596660614014, 60.90747117996216, 62.68668031692505, 63.27247643470764, 62.813350439071655, 64.28944063186646, 63.49366068840027, 62.4254310131073, 62.16382908821106, 64.4770815372467]
test_time: [44.31876826286316, 43.74878478050232, 44.04286527633667, 43.91411733627319, 43.9906485080719, 44.21333885192871, 44.853172063827515, 44.04883646965027, 43.85336637496948, 44.38978552818298]
a

In [18]:
scenario_FKGS(15, 0.5)

len base 0: 1657
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1657
done A
done B
done C
len base 3: 1658
done A
done B
done C
len base 4: 1657
done A
done B
done C
len base 5: 1658
done A
done B
done C
len base 6: 1658
done A
done B
done C
len base 7: 1658
done A
done B
done C
len base 8: 1657
done A
done B
done C
len base 9: 1657
done A
done B
done C
sampling_time: [0.6489951610565186, 0.6373212337493896, 0.632225513458252, 0.6407673358917236, 0.6254062652587891, 0.6302554607391357, 0.6270034313201904, 0.6412158012390137, 0.6412515640258789, 0.6272518634796143]
train_time: [61.84129595756531, 60.2461142539978, 60.127424240112305, 63.18884015083313, 60.98927974700928, 62.411935567855835, 61.50188899040222, 61.14066481590271, 62.32800269126892, 61.509538888931274]
test_time: [44.11719560623169, 43.68656349182129, 43.68556571006775, 44.17387866973877, 43.70164251327515, 44.007105350494385, 43.902119874954224, 43.784417390823364, 43.975284814834595, 44.2624838352

In [19]:
scenario_FKGS(20, 0.1)

len base 0: 2213
done A
done B
done C
len base 1: 2211
done A
done B
done C
len base 2: 2213
done A
done B
done C
len base 3: 2212
done A
done B
done C
len base 4: 2214
done A
done B
done C
len base 5: 2213
done A
done B
done C
len base 6: 2210
done A
done B
done C
len base 7: 2209
done A
done B
done C
len base 8: 2209
done A
done B
done C
len base 9: 2213
done A
done B
done C
sampling_time: [0.8307890892028809, 0.803164005279541, 0.8379724025726318, 0.7974789142608643, 0.8044016361236572, 0.8116264343261719, 0.8085720539093018, 0.8003549575805664, 0.8004634380340576, 0.8061115741729736]
train_time: [119.6720597743988, 115.91049909591675, 124.05732703208923, 118.38921523094177, 119.2368049621582, 122.90708589553833, 120.19425940513611, 118.90028190612793, 118.75467848777771, 122.58953976631165]
test_time: [59.769004583358765, 60.116567850112915, 60.11462879180908, 59.45116925239563, 59.5266695022583, 59.94216012954712, 59.74978566169739, 59.57055902481079, 59.591814279556274, 59.852872

In [20]:
scenario_FKGS(20, 0.2)

len base 0: 2210
done A
done B
done C
len base 1: 2209
done A
done B
done C
len base 2: 2211
done A
done B
done C
len base 3: 2209
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2211
done A
done B
done C
len base 6: 2210
done A
done B
done C
len base 7: 2210
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [0.8962175846099854, 0.8658769130706787, 0.8710877895355225, 0.867173433303833, 0.8888309001922607, 0.8909718990325928, 0.8790349960327148, 0.883105993270874, 0.8790216445922852, 0.8889346122741699]
train_time: [101.23665761947632, 98.03176927566528, 100.72950720787048, 98.78542828559875, 99.37514996528625, 99.36564922332764, 101.22932076454163, 98.91644430160522, 98.59840154647827, 101.01495361328125]
test_time: [57.76774740219116, 57.35410261154175, 57.5030152797699, 57.25601887702942, 57.42631983757019, 57.57127618789673, 57.741477966308594, 57.38291597366333, 57.45953106880188, 57.5502240657806

In [21]:
scenario_FKGS(20, 0.3)

len base 0: 2210
done A
done B
done C
len base 1: 2209
done A
done B
done C
len base 2: 2209
done A
done B
done C
len base 3: 2210
done A
done B
done C
len base 4: 2211
done A
done B
done C
len base 5: 2210
done A
done B
done C
len base 6: 2211
done A
done B
done C
len base 7: 2210
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2210
done A
done B
done C
sampling_time: [1.0286221504211426, 1.0002577304840088, 1.0483169555664062, 1.0231475830078125, 1.0313658714294434, 1.0370380878448486, 1.0341770648956299, 1.0218782424926758, 1.012871265411377, 1.0062541961669922]
train_time: [106.72265815734863, 103.16801905632019, 104.92935943603516, 104.83183693885803, 105.20910739898682, 104.20165252685547, 103.85228872299194, 104.44775295257568, 105.14485836029053, 104.64206647872925]
test_time: [58.40719151496887, 58.163477659225464, 58.195932149887085, 58.19580602645874, 58.211690187454224, 58.2724027633667, 58.256877422332764, 58.258501052856445, 58.3908257484436, 58.189

In [22]:
scenario_FKGS(20, 0.4)

len base 0: 2209
done A
done B
done C
len base 1: 2209
done A
done B
done C
len base 2: 2209
done A
done B
done C
len base 3: 2209
done A
done B
done C
len base 4: 2209
done A
done B
done C
len base 5: 2209
done A
done B
done C
len base 6: 2210
done A
done B
done C
len base 7: 2210
done A
done B
done C
len base 8: 2209
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [1.14100980758667, 1.0993552207946777, 1.1282060146331787, 1.1222562789916992, 1.1068730354309082, 1.1241247653961182, 1.1211247444152832, 1.111158847808838, 1.1373188495635986, 1.113649845123291]
train_time: [112.99997901916504, 113.50677609443665, 111.69939041137695, 111.90276408195496, 112.42398309707642, 112.89681935310364, 109.77503204345703, 112.92614650726318, 111.22968649864197, 110.81191730499268]
test_time: [59.260886669158936, 59.3035786151886, 59.197561502456665, 59.074023723602295, 59.11533761024475, 59.05918550491333, 59.0876727104187, 59.20764994621277, 59.22198724746704, 58.97455501

In [23]:
scenario_FKGS(20, 0.5)

len base 0: 2209
done A
done B
done C
len base 1: 2209
done A
done B
done C
len base 2: 2210
done A
done B
done C
len base 3: 2209
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2210
done A
done B
done C
len base 6: 2210
done A
done B
done C
len base 7: 2209
done A
done B
done C
len base 8: 2209
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [1.1636238098144531, 1.1330525875091553, 1.1233837604522705, 1.1371262073516846, 1.1102137565612793, 1.1333847045898438, 1.117680549621582, 1.1329867839813232, 1.1107606887817383, 1.1150774955749512]
train_time: [109.28297591209412, 110.71247458457947, 109.13541603088379, 113.81412315368652, 109.53041863441467, 111.41848754882812, 109.57314825057983, 108.08012700080872, 110.99267029762268, 109.68560004234314]
test_time: [58.79610252380371, 59.104170083999634, 58.88734698295593, 59.35118365287781, 60.722766637802124, 59.044344902038574, 58.81419277191162, 58.59221696853638, 59.00256538391113, 58.953

In [24]:
scenario_random(10)

len base 0: 1104
done A
done B
done C
len base 1: 1104
done A
done B
done C
len base 2: 1104
done A
done B
done C
len base 3: 1104
done A
done B
done C
len base 4: 1104
done A
done B
done C
len base 5: 1104
done A
done B
done C
len base 6: 1104
done A
done B
done C
len base 7: 1104
done A
done B
done C
len base 8: 1104
done A
done B
done C
len base 9: 1104
done A
done B
done C
sampling_time: [0.007071495056152344, 0.0009996891021728516, 0.0010614395141601562, 0.0010275840759277344, 0.0010001659393310547, 0.0010001659393310547, 0.0, 0.0010006427764892578, 0.0, 0.0009999275207519531]
train_time: [29.4393367767334, 29.743319034576416, 30.12087321281433, 30.048258781433105, 30.889623641967773, 28.99268388748169, 30.495869159698486, 29.585009574890137, 29.44188356399536, 29.9877769947052]
test_time: [29.692627668380737, 29.581331968307495, 29.815850257873535, 29.583397388458252, 29.926159620285034, 29.392134189605713, 29.78689169883728, 29.578551292419434, 29.46222472190857, 29.647089481353

In [25]:
scenario_random(15)

len base 0: 1656
done A
done B
done C
len base 1: 1656
done A
done B
done C
len base 2: 1656
done A
done B
done C
len base 3: 1656
done A
done B
done C
len base 4: 1656
done A
done B
done C
len base 5: 1656
done A
done B
done C
len base 6: 1656
done A
done B
done C
len base 7: 1656
done A
done B
done C
len base 8: 1656
done A
done B
done C
len base 9: 1656
done A
done B
done C
sampling_time: [0.0, 0.0009992122650146484, 0.0009999275207519531, 0.0010113716125488281, 0.0009999275207519531, 0.001008749008178711, 0.0009999275207519531, 0.0, 0.0, 0.0]
train_time: [67.07745003700256, 68.23239588737488, 67.26925921440125, 67.91231513023376, 68.6445848941803, 68.01768565177917, 67.26273155212402, 65.05992841720581, 66.75350522994995, 66.78375673294067]
test_time: [44.78055167198181, 45.05278158187866, 46.26005697250366, 44.84882140159607, 45.031394720077515, 44.92598867416382, 44.715444803237915, 44.27052879333496, 44.62315917015076, 44.49851155281067]
accuracy: [69.71, 69.94, 70.19, 69.75, 69

In [26]:
scenario_random(20)

len base 0: 2208
done A
done B
done C
len base 1: 2208
done A
done B
done C
len base 2: 2208
done A
done B
done C
len base 3: 2208
done A
done B
done C
len base 4: 2208
done A
done B
done C
len base 5: 2208
done A
done B
done C
len base 6: 2208
done A
done B
done C
len base 7: 2208
done A
done B
done C
len base 8: 2208
done A
done B
done C
len base 9: 2208
done A
done B
done C
sampling_time: [0.0010001659393310547, 0.0009996891021728516, 0.0010018348693847656, 0.0010004043579101562, 0.0009999275207519531, 0.0009987354278564453, 0.002154827117919922, 0.0009999275207519531, 0.0010027885437011719, 0.0009999275207519531]
train_time: [120.00812578201294, 119.10279488563538, 120.7384729385376, 119.06341314315796, 120.95114326477051, 119.47748827934265, 117.61793780326843, 119.00171089172363, 118.39196586608887, 122.65717315673828]
test_time: [59.960928440093994, 59.656168699264526, 59.864737033843994, 59.46722412109375, 59.85838985443115, 59.3658971786499, 59.49030017852783, 59.6270740032196